In [1]:
%matplotlib inline
%pylab inline
%load_ext autotime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns
import os
import glob
import geopandas as gpd
import geopy
import plotly_express as px
import tqdm

from graphics import Point
from functools import reduce
from datetime import date
from datetime import datetime
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm.notebook import tqdm_notebook


#from dtale.views import startup
#import dtale
#import pyplot as plt
# os and glob help concat multiple csv files into one

Populating the interactive namespace from numpy and matplotlib
time: 1.45 s (started: 2021-12-20 00:12:10 -06:00)


In [2]:
all_filenames = [i for i in glob.glob('../data/redfindata/*.{}'.format('csv'))]

time: 0 ns (started: 2021-12-20 00:12:11 -06:00)


In [3]:
print(all_filenames)

['../data/redfindata\\RedfinData1.csv', '../data/redfindata\\RedfinData2.csv']
time: 0 ns (started: 2021-12-20 00:12:11 -06:00)


In [4]:
redfin_months = pd.concat([pd.read_csv(f) for f in all_filenames ])

time: 16 ms (started: 2021-12-20 00:12:12 -06:00)


In [5]:
redfin_months.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 700 entries, 0 to 349
Data columns (total 27 columns):
 #   Column                                                                                       Non-Null Count  Dtype  
---  ------                                                                                       --------------  -----  
 0   SALE TYPE                                                                                    700 non-null    object 
 1   SOLD DATE                                                                                    600 non-null    object 
 2   PROPERTY TYPE                                                                                700 non-null    object 
 3   ADDRESS                                                                                      700 non-null    object 
 4   CITY                                                                                         700 non-null    object 
 5   STATE OR PROVINCE                   

In [6]:
#drops columns that have only NAN values
redfin_months.dropna(how='all', axis=1, inplace=True)

time: 0 ns (started: 2021-12-20 00:12:13 -06:00)


In [7]:
#dropped all trash columns 
redfin_months = redfin_months.drop(['SALE TYPE', '$/SQUARE FEET', 'STATUS', 'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)', 'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED', 'LOT SIZE', 'YEAR BUILT', 'HOA/MONTH', 'PROPERTY TYPE'], axis=1)

time: 0 ns (started: 2021-12-20 00:12:13 -06:00)


In [8]:
redfin_months = redfin_months.rename(columns={'ADDRESS': 'Street'})
redfin_months = redfin_months.rename(columns={'STATE OR PROVINCE': 'State'})
redfin_months = redfin_months.rename(columns={'ZIP OR POSTAL CODE': 'Zipcode'})
redfin_months = redfin_months.rename(columns={'SQUARE FEET': 'sqft'})
redfin_months = redfin_months.rename(columns={'BEDS': 'Bedroom'})
redfin_months = redfin_months.rename(columns={'BATHS': 'Bathroom'})
redfin_months = redfin_months.rename(columns={'LATITUDE': 'latitude'})
redfin_months = redfin_months.rename(columns={'LONGITUDE': 'longitude'})

time: 0 ns (started: 2021-12-20 00:12:13 -06:00)


In [9]:
redfin_months.head()

,SOLD DATE,Street,CITY,State,Zipcode,PRICE,Bedroom,Bathroom,LOCATION,sqft,Latitude,Longitude
0,April-20-2021,627 Skyview Dr,Nashville,TN,37206,400000,3.0,1.0,Rolling Acres,1704.0,36.180026,-86.731441
1,May-12-2021,1613 Jacksons Valley Pl,Hermitage,TN,37076,361000,3.0,2.0,Ridgemere,1895.0,36.188682,-86.603974
2,June-3-2021,2076 Stanford Village Dr,Antioch,TN,37013,336000,3.0,2.5,Stanford Village,1641.0,36.021574,-86.689006
3,June-21-2021,1827 Cephas St,Nashville,TN,37208,460000,3.0,2.0,Cephas Woodard,1779.0,36.182059,-86.806558
4,August-5-2021,7620 Staffordshire Dr,Nashville,TN,37221,475000,3.0,2.5,Sheffield On The Harpeth,1843.0,36.064080,-86.971667


time: 15 ms (started: 2021-12-20 00:12:13 -06:00)


In [10]:
all_filenames = [i for i in glob.glob('../data/zillowdata/*.{}'.format('csv'))]

time: 0 ns (started: 2021-12-20 00:12:14 -06:00)


In [11]:
print(all_filenames)

['../data/zillowdata\\zillowdata22.csv', '../data/zillowdata\\zillowfreeapi0.csv', '../data/zillowdata\\zillowfreeapi1.csv', '../data/zillowdata\\zillowsection1.csv', '../data/zillowdata\\zillowsection2.csv', '../data/zillowdata\\zillowsection3.csv', '../data/zillowdata\\zillowsection4.csv', '../data/zillowdata\\zillowwebscrape2.csv', '../data/zillowdata\\zillowwebscrape3.csv', '../data/zillowdata\\zillowzoomed1.csv']
time: 0 ns (started: 2021-12-20 00:12:14 -06:00)


In [12]:
zillow_months = pd.concat([pd.read_csv(f) for f in all_filenames ])

time: 375 ms (started: 2021-12-20 00:12:14 -06:00)


C:\Users\YASHAC~1\AppData\Local\Temp/ipykernel_6556/2921648921.py:1: DtypeWarning: Columns (5,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96) have mixed types.Specify dtype option on import or set low_memory=False.
  zillow_months = pd.concat([pd.read_csv(f) for f in all_filenames ])


In [13]:
zillow_months.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9970 entries, 0 to 175
Columns: 106 entries, Address to photos/81
dtypes: float64(9), object(97)
memory usage: 8.1+ MB
time: 15 ms (started: 2021-12-20 00:12:15 -06:00)


In [14]:
#drops columns that have only NAN values
zillow_months.dropna(how='all', axis=1, inplace=True)

time: 78 ms (started: 2021-12-20 00:12:15 -06:00)


In [15]:
#dropped all trash columns 
zillow_months = zillow_months.drop(['address/city', 'address/neighborhood', 'address/state', 'address/streetAddress', 'address/subdivision', 'address/zipcode', 'photos/72', 'photos/73', 'photos/74', 'photos/75', 'photos/76', 'photos/77', 'photos/78', 'photos/79', 'photos/80', 'photos/81', 'photos/71', 'photos/70', 'photos/69', 'photos/68', 'photos/67', 'photos/66', 'photos/65', 'photos/64', 'photos/63', 'photos/62', 'photos/61', 'photos/60', 'photos/59', 'photos/58', 'photos/57', 'photos/56', 'photos/55', 'photos/54', 'photos/53', 'photos/52', 'photos/51', 'photos/50', 'photos/49', 'photos/48', 'photos/47', 'photos/46', 'photos/45', 'photos/44', 'photos/43', 'photos/42', 'photos/41', 'photos/40', 'photos/39', 'photos/38', 'photos/37', 'photos/36', 'photos/35', 'photos/34', 'photos/33', 'photos/32', 'photos/31', 'photos/30', 'photos/29', 'photos/28', 'photos/27', 'photos/26', 'photos/25', 'photos/24', 'photos/23', 'photos/22', 'photos/21', 'photos/20', 'photos/19'], axis=1)

time: 16 ms (started: 2021-12-20 00:12:15 -06:00)


In [16]:
#more trash
zillow_months = zillow_months.drop(['photos/12', 'photos/13', 'photos/14', 'photos/15', 'photos/16', 'photos/17', 'photos/18', 'price', 'url', 'yearBuilt', 'currency' ], axis=1)

time: 0 ns (started: 2021-12-20 00:12:16 -06:00)


In [17]:
#...more trash
zillow_months = zillow_months.drop(['bathrooms', 'bedrooms', 'homeStatus', 'photos/0', 'photos/1', 'photos/2', 'photos/3', 'photos/4', 'photos/5', 'photos/6', 'photos/7', 'photos/8', 'photos/9', 'photos/10', 'photos/11', 'livingArea'], axis=1)

time: 0 ns (started: 2021-12-20 00:12:16 -06:00)


In [18]:
zillow_months = zillow_months.rename(columns={'list-card-variable-text': 'Sold'})

time: 0 ns (started: 2021-12-20 00:12:16 -06:00)


In [19]:
zillow_months.head()

,Address,Price,Bedroom,Bathroom,sqft,Sold,description,latitude,longitude
0,"536 Wesley Ave, Nashville, TN 37207","$340,000",2.0,1.0,978,Sold 12/17/2021,NaN,NaN,NaN
1,"3528 Paragon Dr, Nashville, TN 37211","$580,000",4.0,3.0,"2,391",Sold 12/17/2021,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"4002 Burrus St, Nashville, TN 37216","$551,000",3.0,3.0,"1,886",Sold 12/17/2021,NaN,NaN,NaN
4,"9721 Katz Ln, Brentwood, TN 37027","$584,820",4.0,3.0,"2,653",Sold 12/17/2021,NaN,NaN,NaN


time: 16 ms (started: 2021-12-20 00:12:17 -06:00)


In [20]:
# split column and add new columns to df
zillow_months[['Street', 'City', 'State']] = zillow_months['Address'].str.split(',', n = 2, expand=True)

time: 16 ms (started: 2021-12-20 00:12:17 -06:00)


In [21]:
zillow_months.head()

,Address,Price,Bedroom,Bathroom,sqft,Sold,description,latitude,longitude,Street,City,State
0,"536 Wesley Ave, Nashville, TN 37207","$340,000",2.0,1.0,978,Sold 12/17/2021,NaN,NaN,NaN,536 Wesley Ave,Nashville,TN 37207
1,"3528 Paragon Dr, Nashville, TN 37211","$580,000",4.0,3.0,"2,391",Sold 12/17/2021,NaN,NaN,NaN,3528 Paragon Dr,Nashville,TN 37211
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"4002 Burrus St, Nashville, TN 37216","$551,000",3.0,3.0,"1,886",Sold 12/17/2021,NaN,NaN,NaN,4002 Burrus St,Nashville,TN 37216
4,"9721 Katz Ln, Brentwood, TN 37027","$584,820",4.0,3.0,"2,653",Sold 12/17/2021,NaN,NaN,NaN,9721 Katz Ln,Brentwood,TN 37027


time: 0 ns (started: 2021-12-20 00:12:17 -06:00)


In [22]:
redfin_months.head()

,SOLD DATE,Street,CITY,State,Zipcode,PRICE,Bedroom,Bathroom,LOCATION,sqft,Latitude,Longitude
0,April-20-2021,627 Skyview Dr,Nashville,TN,37206,400000,3.0,1.0,Rolling Acres,1704.0,36.180026,-86.731441
1,May-12-2021,1613 Jacksons Valley Pl,Hermitage,TN,37076,361000,3.0,2.0,Ridgemere,1895.0,36.188682,-86.603974
2,June-3-2021,2076 Stanford Village Dr,Antioch,TN,37013,336000,3.0,2.5,Stanford Village,1641.0,36.021574,-86.689006
3,June-21-2021,1827 Cephas St,Nashville,TN,37208,460000,3.0,2.0,Cephas Woodard,1779.0,36.182059,-86.806558
4,August-5-2021,7620 Staffordshire Dr,Nashville,TN,37221,475000,3.0,2.5,Sheffield On The Harpeth,1843.0,36.064080,-86.971667


time: 16 ms (started: 2021-12-20 00:12:18 -06:00)


In [23]:
#delete whole row based on column value // get rid of non "nashville" cities
#redfin_months = redfin_months[redfin_months.CITY != 'Hermitage']

time: 0 ns (started: 2021-12-20 00:12:18 -06:00)


In [24]:
#merge df's    Zillow listings and Redfin Listings = join on 
sold_merged_listings = pd.merge(redfin_months, zillow_months, on = 'Street', how='outer')

time: 16 ms (started: 2021-12-20 00:12:19 -06:00)


In [25]:
sold_merged_listings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10665 entries, 0 to 10664
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   SOLD DATE    600 non-null    object 
 1   Street       2002 non-null   object 
 2   CITY         700 non-null    object 
 3   State_x      700 non-null    object 
 4   Zipcode      700 non-null    float64
 5   PRICE        700 non-null    float64
 6   Bedroom_x    569 non-null    float64
 7   Bathroom_x   573 non-null    float64
 8   LOCATION     600 non-null    object 
 9   sqft_x       581 non-null    float64
 10  Latitude     700 non-null    float64
 11  Longitude    700 non-null    float64
 12  Address      1309 non-null   object 
 13  Price        1309 non-null   object 
 14  Bedroom_y    1309 non-null   object 
 15  Bathroom_y   1309 non-null   object 
 16  sqft_y       1309 non-null   object 
 17  Sold         1309 non-null   object 
 18  description  8586 non-null   object 
 19  lati

In [26]:
#drop repeats - unusable columns
sold_merged_listings = sold_merged_listings.drop(['State_x', 'LOCATION', 'description', 'State_y'], axis=1)

time: 16 ms (started: 2021-12-20 00:12:20 -06:00)


In [27]:
sold_merged_listings.tail()

,SOLD DATE,Street,CITY,Zipcode,PRICE,Bedroom_x,Bathroom_x,sqft_x,Latitude,Longitude,Address,Price,Bedroom_y,Bathroom_y,sqft_y,Sold,latitude,longitude,City
10660,NaN,4439 Andrew Jackson Pkwy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"4439 Andrew Jackson Pkwy, Hermitage, TN 37076","$279,900",3,2,"1,988",Sold 01/07/2021,NaN,NaN,Hermitage
10661,NaN,1608 Pakenhams Retreat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1608 Pakenhams Retreat, Hermitage, TN 37076","$348,000",3,3,"2,290",Sold 01/07/2021,NaN,NaN,Hermitage
10662,NaN,1912 Brookmeadow Ct,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1912 Brookmeadow Ct, Hermitage, TN 37076","$275,000",3,2,"1,374",Sold 01/07/2021,NaN,NaN,Hermitage
10663,NaN,626 Albany Dr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"626 Albany Dr, Hermitage, TN 37076","$310,000",3,2,"1,558",Sold 01/06/2021,NaN,NaN,Hermitage
10664,NaN,3809 Plantation Dr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"3809 Plantation Dr, Hermitage, TN 37076","$200,000",5,2,"1,738",Sold 01/06/2021,NaN,NaN,Hermitage


time: 16 ms (started: 2021-12-20 00:12:23 -06:00)


In [28]:
sold_merged_listings.to_csv("RedfinandZillowdata.csv", sep=',', index=False, header=True)

time: 78 ms (started: 2021-12-20 00:19:57 -06:00)


In [ ]:
#used: https://console.apify.com/
#used: APP - Instant Data Scraper (had many limitations)